<a href="https://colab.research.google.com/github/dcpetit/Astronomy/blob/main/Thesis_data_plots_SNRatioPynPoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Plot the data obtained from the S/N tables generated from PynPoint

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [8]:
import pandas as pd
import numpy as np
import scipy.io
from astropy.io import fits
import matplotlib.pyplot as plt
#plt.style.use('ggplot')
font = {#'family' : 'normal',
        #'color' : 'white',
        #'weight' : 'bold',
        'size'   : 11}                          
plt.rc('font', **font)
path = '/content/drive/MyDrive/thesis_masters/2020-02-08_results/'
file2= 'ff_variable-table_properties.txt' # file2's contents are: (3, 3, 39, 200, 200)

g_drive_path = '/content/drive/MyDrive/thesis_masters/'
local_path = '/home/dcpetit/Documents/kuleuven_astronomy/Thesis_masters/beta_pic_results_made6Apr2022/'
yr_2020_path = '2020-02-08_results/ff_variable-table_to_file_'
yr_2018_path = '2018-12-15_results/ff_variable-table_to_file_'
yr_2016_path = '2016-11-18_results/ff_variable-table_to_file_'
yr_2015_path = '2015-11-30_results/ff_variable-table_to_file_'
#df = np.load(g_drive_path+'2020-02-08_results/ff_variable-table_to_file.npy')
#df_SADI_1_3    = np.load('/home/dcpetit/Documents/kuleuven_astronomy/Thesis_masters/beta_pic_results_made6Apr2022/2020-02-08_results/ff_variable-table_to_file_SADI_123123.npy')
df_SADI_1_3    = np.load(local_path+yr_2020_path+'SADI_123123.npy')
df_ASDI_1_3    = np.load(local_path+yr_2020_path+'ASDI_123123.npy')
df_CODI        = np.load(local_path+yr_2020_path+'CODI_100.npy')
df_ASDI_1_6    = np.load(local_path+yr_2020_path+'ASDI_123456123456.npy')
df_SADI_1_6    = np.load(local_path+yr_2020_path+'SADI_123456123456.npy')
df_ASDI_1_8    = np.load(local_path+yr_2020_path+'ASDI_18_18.npy')
df_SADI_1_8    = np.load(local_path+yr_2020_path+'SADI_18_18.npy')
df_ASDI_1_8_2018 = np.load(local_path+yr_2018_path+'ASDI_18.npy')
df_SADI_1_8_2018 = np.load(local_path+yr_2018_path+'SADI_18.npy')
df_ASDI_1_8_2016 = np.load(local_path+yr_2016_path+'ASDI_18.npy')
df_SADI_1_8_2016 = np.load(local_path+yr_2016_path+'SADI_18.npy')
df_ASDI_1_8_2015 = np.load(local_path+yr_2015_path+'ASDI_18.npy')
df_SADI_1_8_2015 = np.load(local_path+yr_2015_path+'SADI_18.npy')
df = df_SADI_1_3
#print(df_CODI)

#file = 'SNR_FormattedByHand.csv'
#fileGood = 'ff_variable-table_to_file.txt'
#df = pd.read_csv(path+file, header=None)#, sep=',') # Now obsolete...?
#df = np.load(path+'ff_variable-table_to_file.npy')

FileNotFoundError: ignored

In [ ]:
'''
# Put this in the first cell eventually...
path_2018 = '/home/dcpetit/Documents/kuleuven_astronomy/Thesis_masters/beta_pic_results_made6Apr2022/2018-12-15_results/' ###
file_2018 = 'SNR_FormattedByHand.csv'
#file_2018 = 'ff_variable-table_to_file.txt'
file2_2018= 'ff_variable-table_properties.txt'

path_2016 = '/home/dcpetit/Documents/kuleuven_astronomy/Thesis_masters/beta_pic_results_made6Apr2022/2016-11-18_results/' ###
file_2016 = 'SNR_FormattedByHand.csv'
#file_2016 = 'ff_variable-table_to_file.txt'
file2_2016= 'ff_variable-table_properties.txt'

path_2015 = '/home/dcpetit/Documents/kuleuven_astronomy/Thesis_masters/beta_pic_results_made6Apr2022/2015-11-30_results/' ###
file_2015 = 'SNR_FormattedByHand.csv'
#file_2015 = 'ff_variable-table_to_file.txt'
file2_2015= 'ff_variable-table_properties.txt'

df_2018 = pd.read_csv(path_2018+file, header=None) #change file name?
df_2016 = pd.read_csv(path_2016+file, header=None)
df_2015 = pd.read_csv(path_2015+file, header=None)

df2_2018= pd.read_table(path_2018+file2, header=None, delimiter=',')
df2_2016= pd.read_table(path_2016+file2, header=None, delimiter=',')
df2_2015= pd.read_table(path_2015+file2, header=None, delimiter=',')

PCAs1_2018 = int(df2_2018[0][0][1])
PCAs1_2016 = int(df2_2016[0][0][1])
PCAs1_2015 = int(df2_2015[0][0][1])

PCAs2_2018 = int(df2_2018[1][0])
PCAs2_2016 = int(df2_2016[1][0])
PCAs2_2015 = int(df2_2015[1][0])

wavelength_index_2018 = int(df2_2018[2][0])
wavelength_index_2016 = int(df2_2016[2][0])
wavelength_index_2015 = int(df2_2015[2][0])

SN_table_2018 = np.zeros( ( PCAs1_2018, PCAs2_2018, wavelength_index_2018 ), dtype=float, order='C')
SN_table_2016 = np.zeros( ( PCAs1_2016, PCAs2_2016, wavelength_index_2016 ), dtype=float, order='C')
SN_table_2015 = np.zeros( ( PCAs1_2015, PCAs2_2015, wavelength_index_2015 ), dtype=float, order='C')

for i in range(11): # 11 columns in the file
    for k in range(39): # 39 rows in the file
        if i == 0 or i == 1 or i == 2:
            SN_table_2018[i, 0, k] = df_2018[i][k]
            ...
        #else:
            #print('i != 0,1,2, 4,5,6, 8,9,10')
'''

In [ ]:
# extract number of images taken in an observation (angles fits file) to calculate Model Completenesses
fits_FilePathFolder = '/home/dcpetit/Documents/kuleuven_astronomy/Thesis_masters/beta_pic_results_made6Apr2022/2020-02-08_results/'  ########
fits_FilePath = fits_FilePathFolder+'angles.fits'
fits_FilePath  = '/content/drive/MyDrive/thesis_masters/2020-02-08_results/angles.fits'
#fits_FilePath = '/home/dcpetit/Documents/kuleuven_astronomy/Thesis_masters/51Eri_28Nov2019/data.fits' #####
#print(fits_FilePath)
fits_image_filename = fits.open(fits_FilePath)
print(fits_image_filename.info())
print('\n', fits_image_filename)
#print('\n', fits_image_filename.info(0))
#print(fits_image_filename.info(0)[0][4])           # this is value of 'cards' (5, this time)
print('\n', fits_image_filename.info(0)[0][5][0])   # this is dimensions/number of exposures/images 
#print('\n', fits_image_filename[0].data)
#print('the length of the header is', len(fits_image_filename[0].header))
print('\n', fits_image_filename[0].header)
#print(fits_image_filename.info(0)[0][5][2])
#print(fits_image_filename.info(0)[0][5][3])
                                                    #hdul = fits.open(fits_image_filename)
N_tot_lambda = int(fits_image_filename.info(0)[0][5][0])
PC_max_SDI = 39-1 #or is it 38?
PC_max_ADI = N_tot_lambda - 1 # = the number of images - 1
PC_max_SADI = 39-1
PC_max_ASDI = 10 # WHAT IS THIS ONE???????
PC_max_CODI = (N_tot_lambda*39) - 1 # = PC_max_ADI * PC_max_SDI

In [ ]:
# Use ADI_wave_median.fits to access data... (it should have a large 200x200x39xpca1xpca2 matrix)
# Then plot it with code below 

image_file = '/content/drive/MyDrive/thesis_masters/2020-02-08_results/ADI_wave_median.fits'
image_file_2018 = '/content/drive/MyDrive/thesis_masters/2018-12-15_results/ADI+SDI_wave_median.fits'
#image_file_2016 = '/content/drive/MyDrive/thesis_masters/2016-11-18_results/ADI_wave_median.fits'
image_file_2015 = '/content/drive/MyDrive/thesis_masters/2015-11-30_results/ADI+SDI_wave_median.fits'
fits_image_filename = fits.open('/content/drive/MyDrive/thesis_masters/2020-02-08_results/ADI_wave_median.fits')

fits.info(image_file)
image_data = fits.getdata(image_file, ext=0)
image_data_2018 = fits.getdata(image_file_2018, ext=0)
#image_data_2016 = fits.getdata(image_file_2016, ext=0)
image_data_2015 = fits.getdata(image_file_2015, ext=0)
#image_data = fits.getdata(fits_image_filename, ext=0) 

# The data is now stored as a 2D numpy array. Print the dimensions using the shape attribute:

print(image_data.shape)
print(image_data_2018.shape)
#print(image_data_2016.shape)
print(image_data_2015.shape)
#print('finding 200 x values', image_data[0][0][0])
print('finding the signal strength in pixel (0,0):', image_data[0][0][0][195])
x, y = image_data[0][0][0], image_data[0][0][1]
x_2018, y_2018 = image_data_2018[0][0][0], image_data_2018[0][0][1]
#x_2016, y_2016 = image_data_2016[0][0][0], image_data_2016[0][0][1]
x_2015, y_2015 = image_data_2015[0][0][0], image_data_2015[0][0][1]

print('length of x and y are,', len(x), len(y))

#print('\n', len(fits_image_filename[0].data))
#print('sum is,', np.sum(fits_image_filename[0].data))
#print('\n', fits_image_filename[0].data)

signalMatrix, signalMatrix_2018, signalMatrix_2016, signalMatrix_2015 = [], [], [], []
sigMatYrs = [signalMatrix, signalMatrix_2018, signalMatrix_2016, signalMatrix_2015] #year 2020 gets no suffix
signalMatrix = np.zeros(200*200).reshape(200,200)
signalMatrix_2018 = np.zeros(200*200).reshape(200,200)
signalMatrix_2016 = np.zeros(200*200).reshape(200,200)
signalMatrix_2015 = np.zeros(200*200).reshape(200,200)
#print(signalMatrix)
for i in range(len(x)): # i --> j
    for j in range(len(y)): # j --> k
        #print('i and j are:\n', i, j)
        signalMatrix[i][j] = image_data[0][32][i][j] # This is ADI... so 1 PCA column...
        signalMatrix_2018[i][j] = image_data_2018[0][0][32][i][j] # This is ASDI or so, so 2!
        #signalMatrix_2016[i][j] = image_data_2016[0][0][32][i][j]
        signalMatrix_2015[i][j] = image_data_2015[0][0][32][i][j]
print('The maximum signal in this (2020) matrix is:', round(max(signalMatrix.ravel()),2))
print('The maximum signal in the 2018 matrix is:', round(max(signalMatrix_2018.ravel()),2))

# Find the brightest pixel's (planet center) coordinates
for i in (range(len(signalMatrix.ravel()))):
    if (signalMatrix.ravel()[i]) > 7.44:
        ravelIndex = i
print('The location of the (2020) max is:', np.unravel_index(ravelIndex,(200,200)))

'''
range(len(sigMatYrs)):
    print('in for loop, sigMatYrs[i] is initially', sigMatYrs[i])
    sigMatYrs[i] = np.zeros(200*200).reshape(200,200)
    print('in for loop, sigMatYrs[i] is finally', sigMatYrs[i])
'''

# Plot the FITS file image

In [ ]:
def plot_FITS_image(year, signalMatrix):
    plt.figure(figsize=(6,6)) #plt.imshow(image_data, cmap='gray') 
    plt.imshow(signalMatrix, cmap='gray', origin='lower') # 'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic
    plt.xlabel('x'), plt.ylabel('y'), plt.title(str(year)+' FITS image (for 33rd WLD)')
    plt.colorbar()
    plt.show()
plot_FITS_image(2020, signalMatrix)
plot_FITS_image(2018, signalMatrix_2018)
#plot_FITS_image(2016, signalMatrix_2016)
plot_FITS_image(2015, signalMatrix_2015)

In [ ]:
#df2= pd.read_table(path+file2, header=None, delimiter=',') # delimiter={',', '(', ')'}
#pd.read_table(path+'ff_variable-table_properties_SADI_18_18.txt', header=None, delimiter=','))
df2 = pd.DataFrame([3, 3, 39, 200, 200]) # ASDI or SADI
df3 = pd.DataFrame([3, 3, 39, 200, 200]) # ASDI or SADI
df4 = pd.DataFrame([100, 39, 200, 200])  # CODI
df_table_ASDI_1_6 = pd.DataFrame([6, 6, 39, 200, 200])
df_table_SADI_1_6 = pd.DataFrame([6, 6, 39, 200, 200])
df_table_ASDI_1_8 = pd.DataFrame([8, 8, 39, 200, 200])
df_table_SADI_1_8 = pd.DataFrame([8, 8, 39, 200, 200]) # same as 2nd line in cell

'''
PCAs1 = int(df2[0][0][1])
PCAs2 = int(df2[1][0])
wavelength_index = int(df2[2][0])
'''
PCAs1 = df2[0][0]
PCAs2 = df2[0][1]
wavelength_index = df2[0][2]
PCAs1_b = df3[0][0]
PCAs2_b = df3[0][1]
wavelength_index_b = df3[0][2]
PCAs1_c = df4[0][0]
#PCAs2_c = df4[0][1]
wavelength_index_c = df4[0][1]

PCAs1_ASDI_1_6 = df_table_ASDI_1_6[0][0]
PCAs2_ASDI_1_6 = df_table_ASDI_1_6[0][1]
wavelength_index_ASDI_1_6 = df_table_ASDI_1_6[0][2]
PCAs1_SADI_1_6 = df_table_SADI_1_6[0][0]
PCAs2_SADI_1_6 = df_table_SADI_1_6[0][1]
wavelength_index_SADI_1_6 = df_table_SADI_1_6[0][2]

PCAs1_ASDI_1_8 = df_table_ASDI_1_8[0][0]
PCAs2_ASDI_1_8 = df_table_ASDI_1_8[0][1]
wavelength_index_ASDI_1_8 = df_table_ASDI_1_8[0][2]
PCAs1_SADI_1_8 = df_table_SADI_1_8[0][0]
PCAs2_SADI_1_8 = df_table_SADI_1_8[0][1]
wavelength_index_SADI_1_8 = df_table_SADI_1_8[0][2]

'''print('PCAs1_ASDI_1_6 is:', PCAs1_ASDI_1_6)
print('PCAs2_ASDI_1_6 is:', PCAs2_ASDI_1_6) 
print('wavelength_index_ASDI_1_6 is:', wavelength_index_ASDI_1_6) 
print('PCAs1_c is:',PCAs1_c)
print('wavelength_index_c is:',wavelength_index_c)
print(df_ASDI_1_6)'''

# A for loop to organize the data_frame into a plotable SNRatios
SN_table = np.zeros( ( PCAs1, PCAs2, wavelength_index ), dtype=float, order='C') #print(SN_table)
SN_table_b = np.zeros( ( PCAs1_b, PCAs2_b, wavelength_index_b ), dtype=float, order='C') #print(SN_table)
SN_table_c = np.zeros( ( PCAs1_c, wavelength_index_c ), dtype=float, order='C') #print(SN_table)
SN_table_ASDI_1_6 = np.zeros( ( PCAs1_ASDI_1_6, PCAs2_ASDI_1_6, wavelength_index_ASDI_1_6 ), dtype=float, order='C') #print(SN_table)
SN_table_SADI_1_6 = np.zeros( ( PCAs1_SADI_1_6, PCAs2_SADI_1_6, wavelength_index_SADI_1_6 ), dtype=float, order='C') #print(SN_table)
SN_table_ASDI_1_8 = np.zeros( ( PCAs1_ASDI_1_8, PCAs2_ASDI_1_8, wavelength_index_ASDI_1_8 ), dtype=float, order='C') #print(SN_table)
SN_table_SADI_1_8 = np.zeros( ( PCAs1_SADI_1_8, PCAs2_SADI_1_8, wavelength_index_SADI_1_8 ), dtype=float, order='C') #print(SN_table)
model_completeness = np.zeros( ( PCAs1, PCAs2, 2 ), dtype=float, order='C')
#print('the SN table is \n', SN_table)
bad_character = [ '[', ' ', ',', ']' ]
# (i+1)/(39-1) is for SDI
# (j+1)/(N_tot_lambda - 1) is for ADI
for i in range(3): 
    for j in range(3):
        for k in range(39): # 39 rows in the file/data
            SN_table[i, j, k] = df[i][j][k]
            model_completeness[i, j, :] = [(i+1)/(N_tot_lambda - 1), (j+1)/(39-1)] # for ASDI (or maybe SADI)

for i in range(3): 
    for j in range(3):
        for k in range(39): # 39 rows in the file/data
            SN_table_b[i, j, k] = df_ASDI_1_3[i][j][k]
            model_completeness[i, j, :] = [(i+1)/(N_tot_lambda - 1), (j+1)/(39-1)] # for SADI (or maybe ASDI)


for i in range(3): 
    for j in range(3):
        for k in range(39): # 39 rows in the file/data
            SN_table_ASDI_1_6[i, j, k] = df_ASDI_1_6[i][j][k]
            model_completeness[i, j, :] = [(i+1)/(N_tot_lambda - 1), (j+1)/(39-1)]

for i in range(6): 
    for j in range(6):
        for k in range(39): # 39 rows in the file/data
            SN_table_ASDI_1_6[i, j, k] = df_ASDI_1_6[i][j][k]
            #model_completeness[i, j, :] = [(i+1)/(N_tot_lambda - 1), (j+1)/(39-1)]

for i in range(6):
    for j in range(6):
        for k in range(39): 
            SN_table_SADI_1_6[i, j, k] = df_SADI_1_6[i][j][k]

for i in range(8):
    for j in range(8):
        for k in range(39):
            SN_table_ASDI_1_8[i, j, k] = df_ASDI_1_8[i][j][k]

for i in range(8):
    for j in range(8):
        for k in range(39):
            SN_table_SADI_1_8[i, j, k] = df_SADI_1_8[i][j][k]


''' For CODI data reduction: 
print(df_ASDI_1_3)
print(df_CODI)
for i in range(3): 
    for k in range(39): # 39 rows in the file/data
        SN_table_c[i, k] = df_CODI[i][k]
        #model_completeness[i, j, :] = [(i+1)/(N_tot_lambda - 1), (j+1)/(39-1)] # for CODI
        model_completeness[i, j] = [(i+1)/(N_tot_lambda - 1), (j+1)/(39-1)] # for CODI
        # CHANGE THIS MODEL COMPLETENESS           
'''

print('\n\n', model_completeness[1,1])
print('\n\n', model_completeness)
print('\n\n', SN_table[0][0][0])
#print('\n\n', SN_table[0][0])
#print('\n\n', SN_table[0])
#print('\n\n', SN_table)

# For now, take the Max(SNR) for each PCA, use that. Later, use the whole function of SNRs
# Can then make heat maps: techniques vs. max(SNR)... x:adi y:sdi
# Use all of the 4 Beta Pic observations

In [ ]:
# Plot things #1: Signal to Noise ratio vs wavelength index, for various PCAs
p = plt.get_cmap('inferno') 
plt.figure(figsize = (20, 6))
wavelength_index = range(len(SN_table[0][0]))
counter_color = 0
for j in range(3):       # switch back to i then j! 
    for i in range(3):
        plt.plot(wavelength_index, SN_table[j][i], color=p(counter_color/((3*3)-1)), label='($PCA_{1,2}$)=('+str(i+1)+','+str(j+1)+')') ###
        #plt.plot(wavelength_index, SN_table[j][i], color=clrs[3*i+j], label='($PCA_{1,2}$)=('+str(i+1)+','+str(j+1)+')') ###
        #plt.plot(rad, sig, linewidth=0.4, color=p(i/(n-1)), label=str(i+1))
        counter_color += 1
plt.xlabel('Wavelength Index (WLI)')
plt.ylabel('Signal to Noise Ratio')
plt.legend()

In [ ]:
# Plot things #1: Signal to Noise ratio vs wavelength index, for various PCAs
p = plt.get_cmap('inferno') 
plt.figure(figsize = (8, 9))
wavelength_index = range(len(SN_table[0][0]))
counter_color = 0
for j in range(3):       # switch back to i then j! 
    for i in range(3):
            plt.plot(wavelength_index, SN_table[j][i], color=p(counter_color/((3*3)-1)), label='($PCA_{1,2}$)=('+str(i+1)+','+str(j+1)+')') ###
            counter_color += 1
plt.xlabel('Wavelength Index (WLI)')
plt.ylabel('Signal to Noise Ratio')
plt.title('Zoomed in on maximum of above')
plt.xlim(30,36)
plt.ylim(0,190)
plt.legend(ncol=2)

In [ ]:
# Plot things #2: Constant PCA1, plot PCA2 on varying wavelenths; constant PCA2, plot PCA1 on varying wavelenths
#        plt.plot(wavelength_index, PCA2, label='PCA_1=1')
#        plt.plot(wavelength_index, PCA1, label='PCA_2=1')

In [ ]:
# This cell defines a function that takes in a SN_Table as input, determines, and outputs the maximum signal strength. It will be used several times in the next cell. Future investigations can expand from the simple maximum to some weighted average or weighted maximum range average. 
def find_maximum_signal(SN_table, PCAs1, PCAs2):
    wavelength_index = range(len(SN_table[0][0]))
    max_signal_strength = np.zeros( ( PCAs1, PCAs2 ), dtype=float, order='C') #might need to have pcas as inputs too
    counter = len(SN_table)
    #print('the counter is', counter)
    for i in range(counter):
        for j in range(counter):
            #print('i is now', i)
            #print('j is now', j)
            max_signal_strength[i,j] = (max(SN_table[j][i]))
            # wavelength_index, SN_table[j][i]
    #print(max_signal_strength) 
    return(max_signal_strength)

#find_maximum_signal(SN_table_ASDI_1_6, 6, 6)

In [ ]:
# Prepare heat map plots' data
max_signal_strength = find_maximum_signal(SN_table, 3, 3)
'''wavelength_index = range(len(SN_table[0][0]))
max_signal_strength = np.zeros( ( PCAs1, PCAs2 ), dtype=float, order='C')
for i in range(3):
    for j in range(3):
        max_signal_strength[i,j] = (max(SN_table[j][i]))
        # wavelength_index, SN_table[j][i]
print(max_signal_strength) 
'''
a = pd.DataFrame(max_signal_strength)       # a = np.random.random((8, 8))
max_signal_strength_b = find_maximum_signal(SN_table_b, 3, 3)
b = pd.DataFrame(max_signal_strength_b)
max_signal_strength_ASDI_1_6 = find_maximum_signal(SN_table_ASDI_1_6, 6, 6)
ASDI_16 = pd.DataFrame(max_signal_strength_ASDI_1_6) #ASDI 1-6
max_signal_strength_SADI_1_6 = find_maximum_signal(SN_table_SADI_1_6, 6, 6)
SADI_16 = pd.DataFrame(max_signal_strength_SADI_1_6) #SADI 1-6

max_signal_strength_ASDI_1_8 = find_maximum_signal(SN_table_ASDI_1_8, 8, 8)
ASDI_18 = pd.DataFrame(max_signal_strength_ASDI_1_8) #ASDI 1-6
max_signal_strength_SADI_1_8 = find_maximum_signal(SN_table_SADI_1_8, 8, 8)
SADI_18 = pd.DataFrame(max_signal_strength_SADI_1_8) #SADI 1-6
# Do I use different days/observations now to compare heat maps?
data = pd.DataFrame([[a],[b],[ASDI_16],[SADI_16],[ASDI_18],[SADI_18]]) # this will be pcas, with [a],[b],[c],[d] 

#Plot the Heat Maps

In [ ]:
# plot HEAT MAPS of average Signal-to-Noise Ratios for the varied PCAs
from mpl_toolkits.axes_grid1 import AxesGrid
plt.rcParams["axes.grid"] = False #removes white line-grid 
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable

# if feasible, use a command in imshow to flip the axis so origin is in bottom left

#fig, [[ax1, ax2], [ax3, ax4]] = plt.subplots(nrows=2, ncols=2, figsize = (14,12) ) 
fig, [[ax1, ax2], [ax3, ax4], [ax5, ax6]] = plt.subplots(nrows=3, ncols=2, figsize = (11.5,15) ) 
#ax = [ax1, ax2, ax3, ax4] # for i in range(4): ax[i]?
ax = [ax1, ax2, ax3, ax4, ax5, ax6]
fig.suptitle('Heat maps of maximum signal to noise ratios wrt PCAs during 2020 observation', fontsize=20)
len_PCAs = [len(a), len(b), len(ASDI_16), len(SADI_16), len(ASDI_18), len(SADI_18)]
print('the range(len(data)) is', np.array(range(len(data))))
print('the len_PCAs is', len_PCAs)
for i in range(len(data)):
    image = ax[i].imshow(data[0][i], cmap='plasma', origin='lower', extent=[0.5, 0.5+int(len_PCAs[i]), 0.5, int(len_PCAs[i])+0.5], rasterized=True)
    divider=make_axes_locatable(ax[i])
    cax = divider.append_axes('right', size='10%', pad=0.4)
    cbar= fig.colorbar(image, cax=cax)
#ticks = ['1','2','3']
#ax1.set_xticks([0,1,2])
ax1.set_xlabel('$PCAs_{ADI\ OR\ SDI}$')
ax1.set_ylabel('$PCAs_{ADI\ OR\ SDI}$')
ax1.set_title('ax1 title (ASDI)')
#ax2.set_xticks([1,2,3])
ax2.set_xlabel('$PCAs_{ADI\ OR\ SDI}$')
ax2.set_ylabel('$PCAs_{ADI\ OR\ SDI}$')
ax2.set_title('ax2 title (SADI)')
#ax3.set_xticks([1,2,3])
ax3.set_xlabel('$PCAs_{CODI}$')
ax3.set_ylabel('$PCAs_{Um}... what goes here?$')
ax3.set_title('ax3 title (CODI)')
#ax4.set_xticks([1,2,3])
ax4.set_xlabel('$X-Label PCAs_{CODI}$')
ax4.set_ylabel('$Y-Label PCAs_{Um}$')
ax4.set_title('ax4 SADI 6 PCAs')
#ax5.set_xticks([1,2,3])
ax5.set_xlabel('$X-Label PCAs_{CODI}$')
ax5.set_ylabel('$Y-Label PCAs_{Um}$')
ax5.set_title('ax5 ASDI 8 PCAs')
#ax6.set_xticks([1,2,3])
ax6.set_xlabel('$X-Label PCAs_{CODI}$')
ax6.set_ylabel('$Y-Label PCAs_{Um}$')
ax6.set_title('ax6 SADI 8 PCAs')
plt.show()

'''
im1 = ax1.imshow(data[0][0], cmap='plasma', origin='lower', interpolation='nearest', rasterized=True)
divider1 = make_axes_locatable(ax1) 
cax1 = divider1.append_axes("right", size="10%", pad=0.4)
cbar1 = fig.colorbar(im1, cax=cax1)
im2 = ax2.imshow(data[0][1], cmap='plasma', origin='lower', interpolation='nearest', rasterized=True)
divider2 = make_axes_locatable(ax2) 
cax2 = divider2.append_axes("right", size="10%", pad=0.4)
cbar2 = fig.colorbar(im2, cax=cax2)
im3 = ax3.imshow(data[0][2], cmap='plasma', origin='lower', interpolation='nearest', rasterized=True)
divider3 = make_axes_locatable(ax3) 
cax3 = divider3.append_axes("right", size="10%", pad=0.4)
cbar3 = fig.colorbar(im3, cax=cax3)
im4 = ax4.imshow(data[0][3], cmap='plasma', origin='lower', interpolation='nearest', rasterized=True)
divider4 = make_axes_locatable(ax4) 
cax4 = divider4.append_axes("right", size="10%", pad=0.4)
cbar4 = fig.colorbar(im4, cax=cax4)
'''

'''
# From stack overflow:
a = np.random.random((16, 16))
#plt.imshow(a, cmap='hot', interpolation='nearest')
#plt.show()
fig = plt.figure()
grid = AxesGrid(fig, 111,
                nrows_ncols=(2, 2),
                axes_pad=0.05,
                share_all=True,
                label_mode="L",
                cbar_location="right",
                cbar_mode="single",
                )
for val, ax in zip(a,grid):
    #im = ax.imshow(val, vmin=0, vmax=1)
    im = ax.imshow(a, cmap='hot', interpolation='nearest')
grid.cbar_axes[0].colorbar(im)
for cax in grid.cbar_axes:
    cax.toggle_label(False)
'''

In [ ]:
'''

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14,8))
fig.suptitle('Signal to Noise Ratios on Waveindex Number for 3 Observations')
for i in range(3):
    for j in range(3):
        ax1.plot(range(len(SN_table_2018[0][0])), SN_table_2018[j][i], color=clrs[3*i+j], label='($PCA_{2018: 1,2}$)=('+str(i+1)+','+str(j+1)+')') ###
        ax2.plot(range(len(SN_table_2016[0][0])), SN_table_2016[j][i], color=clrs[3*i+j], label='($PCA_{2016: 1,2}$)=('+str(i+1)+','+str(j+1)+')') ###
        ax3.plot(range(len(SN_table_2015[0][0])), SN_table_2015[j][i], color=clrs[3*i+j], label='($PCA_{2015: 1,2}$)=('+str(i+1)+','+str(j+1)+')') ###     
        
        # a = np.empty(9)
        # a[j] = i
        # ax.plot(range(9), a, color=greens[i], label='$green_{'+str(i)+'}$')
        
ax1.set_ylim([-4, 5.2])
ax2.set_ylim([-4, 5.2])
ax3.set_ylim([-4, 5.2])
ax2.set_ylabel('Signal to Noise Ratio') #ax1.set_ylabel('Signal to Noise Ratio')
ax3.set_xlabel('Wavelength Index Number (WLI)')
ax1.set_title('2018', fontsize=10)
ax2.set_title('2016', fontsize=10)
ax3.set_title('2015', fontsize=10)
ax1.legend()
ax2.legend()
ax3.legend()
'''

# Mathematics of profile cut plots

In [ ]:
# Determine which pixels are along the line between the star and the planet.
# An ADI image in ds9 suggests: bottom left is origin (0,0), and the top right is (200,200)

#signalMatrix =  # the large matrix of signal strength from the data reduction (image in numbers)
FieldOfView_X = 200 # from the image outputs or from Sven
FieldOfView_Y = 200 # from the image outputs or from Sven
Xcen, Ycen = 137.65, 78.82 #actually Xcen=78.82,Ycen=137.65... # the x & y coordinates are switched 
Xcen_2018, Ycen_2018 = 120.5, 89 # 90, 121 # best might be with 35th WLD and with PCAs=2,2
#Xcen_2016, Ycen_2016 = 137, 78
Xcen_2015, Ycen_2015 = 117, 74.5 # best might be with 34th WLD and with PCAs=2,1
test_Xcen, test_Ycen = 125, 193 # 25, 40

def inputCenter_outputPlot(XCen, YCen):
    pixelSize = 1 # from the image outputs or from Sven
    signalCenter = [XCen, YCen] # from the data reduction's code, or from visual inspection
    m = ((YCen-100)/(XCen-100))
    X_profile_max = (XCen/YCen)*FieldOfView_Y # calculated while assuming the star was at the origin. Otherwise use = ((Xcen-Xstar)/(Ycen-Ystar))*FieldOfView_Y
    if abs(m) > 1:
        X_prof_min_buffer = 100*(1 - (1/abs(m)))
        X_prof_max_buffer = 100*(1 + (1/abs(m))) #abs(m)?
        Y_prof_min_buffer = 0
        Y_prof_max_buffer = 0
        X_profile_min = 0 + X_prof_min_buffer #(Xcen/Ycen)*FieldOfView_Y
        X_profile_max = 200 - X_prof_min_buffer #(Xcen/Ycen)*FieldOfView_Y
    else:
        Y_prof_min_buffer = 100*(1 - (1/abs(m))) #abs(m)?
        Y_prof_max_buffer = 100*(1 + (1/abs(m))) #abs(m)?
        X_prof_min_buffer = 0
        X_prof_max_buffer = 0
        X_profile_min = 0 + X_prof_min_buffer #(Xcen/Ycen)*FieldOfView_Y
        X_profile_max = 200 - X_prof_max_buffer #(Xcen/Ycen)*FieldOfView_Y
    print('X_profile_min: ', X_profile_min)
    print('X_profile_max: ', X_profile_max)
    xComponent = np.linspace(X_profile_min, X_profile_max, 1000) #maybe more than 1000
    yComponent = YCen + m*(xComponent - XCen)
    #print('x is: '+str(xComponent)+'.\n\ny is: '+str(yComponent)) # Test: should be values that start and end between 0 and 200
    plt.plot(xComponent, yComponent, '-', linewidth=3)
    plt.plot(XCen, YCen, 'o')
    plt.plot(100, 100, 'o')
    plt.title('The profile cut for the point ('+str(XCen)+', '+str(YCen)+')')
    plt.xlim(-20,220)
    plt.ylim(-20,220)
    plt.show()
    return(xComponent, m)
xComponent, m = inputCenter_outputPlot(Xcen, Ycen)                      #137.65, 78.82
testX, testm = inputCenter_outputPlot(test_Xcen, test_Ycen)
radiusArray = [] # or rename this "pixelPath"

xComponent_2018, m_2018 = inputCenter_outputPlot(Xcen_2018, Ycen_2018)  #120.5, 89
radiusArray_2018 = []
#xComponent_2016, m_2016 = inputCenter_outputPlot(Xcen_2016, Ycen_2016)
radiusArray_2016 = []
xComponent_2015, m_2015 = inputCenter_outputPlot(Xcen_2015, Ycen_2015)
radiusArray_2015 = []

# This approach determines in which pixel each point on the line is in, and then removes the many redundant pixels
def pixels_under_the_star_planet_line(Xcen, Ycen, xComponent, m):
    boxes = []
    for i in xComponent:
        xCoord = i
        yCoord = Ycen + m*(i - Xcen) #slope of the line equation (Ycen + m*(i-Xcen))???
        boxes.append([int(xCoord), int(yCoord)])
        #print('all boxes are:\n', boxes)
    unique_boxes = []
    for i in boxes:
        if i not in unique_boxes:
            unique_boxes.append(i)
    UniqueBoxesNP = np.array(unique_boxes)
    #print('numpy unique boxes are:\n', UniqueBoxesNP)
    print('len of boxes is:\n', len(boxes)) # should be the number of points in the np.linspace command 
    print('len of numpy unique boxes is:\n', len(UniqueBoxesNP))
    return(UniqueBoxesNP)
UniqueBoxesNP = pixels_under_the_star_planet_line(Xcen, Ycen, xComponent, m)
test_UniBoxNP = pixels_under_the_star_planet_line(test_Xcen, test_Ycen, testX, testm)

UniqueBoxesNP_2018 = pixels_under_the_star_planet_line(Xcen_2018, Ycen_2018, xComponent_2018, m_2018)
#UniqueBoxesNP_2016 = pixels_under_the_star_planet_line(Xcen_2016, Ycen_2016, xComponent_2016, m_2016)
UniqueBoxesNP_2015 = pixels_under_the_star_planet_line(Xcen_2015, Ycen_2015, xComponent_2015, m_2015)

'''
# This approach loops through all pixels and determines if any part of the line is within its boutnds
for i in range(FieldOfView_X):
    for j in range(FieldOfView_Y):
        #print('i is: '+str(i)+', and j is: '+str(j))
        if xComponent.any()>i*pixelSize and xComponent.any()<(i+1)*pixelSize and yComponent.any()>j*pixelSize and yComponent.any()<(j+1)*pixelSize: # Do I want to use a "here < x < there" or a "x in i&j" approach here?
#        if xComponent in i and yComponent in j: # Do I want to use a "here < x < there" or a "x in i&j" approach here?
            radiusArray.append([i,j])
print('radiusArray (unnecessary):\n', (radiusArray))
radiusArrayNP = np.array(radiusArray)
radiusArrayUnique = np.unique(radiusArrayNP)
print('radiusArrayUnique (unnecessary):\n', (radiusArrayUnique))
# Here flatten the array so there are no duplicate values
'''

def signal_of_selected_pixels(UniqueBoxesNP):
    print('length of input:', len(UniqueBoxesNP))
    CrossSectionalSignal = []
    for i in UniqueBoxesNP:
        # print(i)
        if i[0] <= 199.1 and i[1] < 199.1:
            #print('i[0] and i[1]', i[0], i[1])
            #print(signalMatrix[i[0]][i[1]])
            CrossSectionalSignal.append(signalMatrix[i[0]][i[1]])
            #CrossSectionalSignal = signalMatrix[UniqueBoxesNP] # the signal values at coordinates radiusArray
    #print('CrossSSig is: ', CrossSectionalSignal)
    print('CrossSSig Max is: ', max(CrossSectionalSignal))
    for i in range(len(CrossSectionalSignal)):
        if (CrossSectionalSignal[i] + 0.001) > max(CrossSectionalSignal):
            CSS_max_location = i
    print('CrossSSig Max\'s location is: ', CSS_max_location)

    # convert Unique boxes into 'distance from star' array [-100, 100]
    # print(UniqueBoxesNP[:,0])
    DistanceFromStar = np.sqrt((UniqueBoxesNP[:,0] - 100)**2 + (UniqueBoxesNP[:,1] - 100)**2)
    for i in range(int(len(DistanceFromStar)/2)):
        DistanceFromStar[i] = (-1)*DistanceFromStar[i]
    if len(DistanceFromStar) != len(CrossSectionalSignal):
        DistanceFromStarTruncate = np.delete(DistanceFromStar,-1)
    else:
        DistanceFromStarTruncate = DistanceFromStar
    return(CrossSectionalSignal, DistanceFromStarTruncate)
CrossSectionalSignal, DistanceFromStarTruncate = signal_of_selected_pixels(UniqueBoxesNP)
#print(test_UniBoxNP)
testCSS, testDFST = signal_of_selected_pixels(test_UniBoxNP)

CrossSectionalSignal_2018, DistanceFromStarTruncate_2018 = signal_of_selected_pixels(UniqueBoxesNP_2018)
#CrossSectionalSignal_2016, DistanceFromStarTruncate_2016 = signal_of_selected_pixels(UniqueBoxesNP_2016)
CrossSectionalSignal_2015, DistanceFromStarTruncate_2015 = signal_of_selected_pixels(UniqueBoxesNP_2015)


# Plot the signal strength  along the line from the center thorugh the brightest/central planet pixel/signal. Find a way to characterize the darkness/dips before and after the peak signal (the artifact) ##
plt.figure(figsize=(10,4))
plt.plot(DistanceFromStarTruncate, CrossSectionalSignal)
plt.title('Signal on (line) profile of stars & planet (2020 obs)')
plt.xlabel('Distance/radius')
plt.ylabel('Signal strength')
plt.ylim(-5,8)
plt.show()
plt.figure(figsize=(10,4))
plt.plot(testDFST, testCSS)
plt.title('Signal on another (test line) profile through the star')
plt.ylabel('Signal strength')
plt.ylim(-5,8)
plt.show()
plt.figure(figsize=(10,4))
plt.plot(DistanceFromStarTruncate_2018, CrossSectionalSignal_2018)
plt.title('Signal on (line) profile of stars & planet (2018 obs)')
plt.xlabel('Distance/radius')
plt.ylabel('Signal strength')
plt.ylim(-5,8)
plt.show()
'''plt.plot(DistanceFromStarTruncate_2016, CrossSectionalSignal_2016)
plt.title('Signal on (line) profile of stars & planet (2016 obs)')
plt.xlabel('Distance/radius')
plt.ylabel('Signal strength')
plt.ylim(-5,8)
plt.show()'''
plt.plot(DistanceFromStarTruncate_2015, CrossSectionalSignal_2015)
plt.title('Signal on (line) profile of stars & planet (2015 obs)')
plt.xlabel('Distance/radius')
plt.ylabel('Signal strength')
plt.ylim(-5,8)
plt.show()

# After getting this first plot, do it again 39 times for each wavelength_index... 

In [ ]:
# plot the imshow & plt together 
plt.figure(figsize=(11,11)) 
plt.title('SPHERE image with planet-star cut superimposed')
plt.imshow(signalMatrix, cmap='gray', origin='upper') # 'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic
plt.colorbar()
xComponent, m = inputCenter_outputPlot(Xcen, Ycen)
plt.show()

In [ ]:
# this cell turns the (messy) plotting above code into a clean function 
def plot_signal_profile(wavelength_index, UniqBox, img_data_yr):
    '''
    if real_test_planet == 'real':
        UniqueBoxes = UniqueBoxesNP
    elif real_test_planet == 'test':
        UniqueBoxes = test_UniBoxNP
    else:
        print('error in plot signal profile function')
    '''

    DistanceFromStar = np.sqrt((UniqBox[:,0] - 100)**2 + (UniqBox[:,1] - 100)**2)
    # print(DistanceFromStar)
    for i in range(int(len(DistanceFromStar)/2)): #make half the line in a negative direction 
        DistanceFromStar[i] = (-1)*DistanceFromStar[i] 
    radius = np.delete(DistanceFromStar,-1)

    #print(img_data_yr.shape)

    for i in range(200):
        for j in range(200):
            if len(img_data_yr.shape) == 4: # or shape == 4
                signalMatrix[i][j] = img_data_yr[0][wavelength_index][i][j]
            elif len(img_data_yr.shape) == 5: # or shape == 5
                signalMatrix[i][j] = img_data_yr[0][0][wavelength_index][i][j]
            else:
                print('Your input matrix does not have appropriate dimensions... is it ASDI vs. ADI?')

    signal = []
    for i in UniqBox: # might need to define this 
        if i[0] <= 199.1 and i[1] < 199.1:
            signal.append(signalMatrix[i[0]][i[1]])
    return(radius, signal)

# Plot the 39 detectors 

In [ ]:
# this cell makes the above plot for all 39 wavelengths
wavelength_index = range(39)
n = max(wavelength_index)

plt.figure(figsize=(18,10))
for i in wavelength_index:
    rad, sig = plot_signal_profile(i, UniqueBoxesNP, image_data)
    plt.plot(rad, sig, linewidth=0.4, color=p(i/(n-1)), label=str(i+1))
plt.title('Signal to noise on a line through the star and planet for different WLDs (2020)')
plt.xlim(-70, 70)
plt.xlabel('Distance from star [pixels] (+/- 0 is arbitrary)')
plt.ylabel('Signal strength')
plt.legend(ncol=3)
plt.show()

plt.figure(figsize=(18,10))
for i in wavelength_index:
    rad_2018, sig_2018 = plot_signal_profile(i, UniqueBoxesNP_2018, image_data_2018) # Peak should be at ~23 pixels distance?
    plt.plot(rad_2018, sig_2018, linewidth=0.4, color=p(i/(n-1)), label=str(i+1))
plt.title('Signal to noise on a line through the star and planet for different WLDs (2018)'), plt.xlim(-70, 70), 
plt.xlabel('Distance from star [pixels] (+/- 0 is arbitrary)'), plt.ylabel('Signal strength'), plt.legend(ncol=3)
plt.show()
'''plt.figure(figsize=(18,10))
for i in wavelength_index:
    rad_2016, sig_2016 = plot_signal_profile(i, UniqueBoxesNP_2016, image_data_2016)
    plt.plot(rad_2016, sig_2016, linewidth=0.4, color=p(i/(n-1)), label=str(i+1))
plt.title('Signal to noise on a line through the star and planet for different WLDs (2016)'), plt.xlim(-70, 70), 
plt.xlabel('Distance from star [pixels] (+/- 0 is arbitrary)'), plt.ylabel('Signal strength'), plt.legend(ncol=3)
plt.show()'''
plt.figure(figsize=(18,10))
for i in wavelength_index:
    rad_2015, sig_2015 = plot_signal_profile(i, UniqueBoxesNP_2015, image_data_2015)
    plt.plot(rad_2015, sig_2015, linewidth=0.4, color=p(i/(n-1)), label=str(i+1))
plt.title('Signal to noise on a line through the star and planet for different WLDs (2015)'), plt.xlim(-70, 70), 
plt.xlabel('Distance from star [pixels] (+/- 0 is arbitrary)'), plt.ylabel('Signal strength'), plt.legend(ncol=3)
plt.show()

In [ ]:
# Zoom in on the planet signal (hill) around 40 [pixels]
WLI_of_interest = 35 - 1 # leave the -1 for python's indexing
plt.figure(figsize=(10,6))
for i in wavelength_index:
    rad, sig = plot_signal_profile(i, UniqueBoxesNP, image_data)
    if i != WLI_of_interest:
        plt.plot(rad, sig, linewidth=0.4, color=p(i/(n-1)), label=str(i+1))
    else:
        plt.plot(rad, sig, linewidth=4, color=p(i/(n-1)), label=str(i+1))
plt.title('Signal to noise on a line through the planet for different WLDs')
plt.xlim(30, 55)
plt.ylim(-1, 9)
plt.xlabel('Distance from star [pixels]')
plt.ylabel('Signal strength')
plt.legend(ncol=3)
plt.show()

plt.figure(figsize=(6,4))
wl_selection = wavelength_index[30:38]
for i in wl_selection:
    rad, sig = plot_signal_profile(i, UniqueBoxesNP, image_data)
    if i != WLI_of_interest:
        plt.plot(rad, sig, linewidth=1, color=p((i-29)/(len(wl_selection)-1)), label=str(i+1))
    else:
        plt.plot(rad, sig, linewidth=3, color=p((i-29)/(len(wl_selection)-1)), label=str(i+1))
plt.title('Zoom in on peak (colors adapted)')
plt.xlim(40, 44)
plt.ylim(6, 8)
plt.xlabel('Distance from star [pixels]')
plt.ylabel('Signal strength')
plt.legend(ncol=1)
plt.show()

In [ ]:
# Zoom in on 2018 plot
plt.figure(figsize=(18,10))
for i in wavelength_index:
    rad_2018, sig_2018 = plot_signal_profile(i, UniqueBoxesNP_2018, image_data_2018) # Peak should be at ~23 pixels distance?
    plt.plot(rad_2018, sig_2018, linewidth=0.4, color=p(i/(n-1)), label=str(i+1))
plt.title('ZOOMED Signal to noise on a line through the star and planet for different WLDs (2018)'), 
plt.xlabel('Distance from star [pixels] (+/- 0 is arbitrary)'), plt.ylabel('Signal strength'), plt.legend(ncol=3)
plt.xlim(15,30)
plt.ylim(-4,2.5)
plt.show()

In [ ]:
# plot the s/r through a random cut profile (with no planet) 
wavelength_index = range(39)
n = max(wavelength_index)
p = plt.get_cmap('inferno') 
plt.figure(figsize=(15,8))
for i in wavelength_index:
    rad, sig = plot_signal_profile(i, test_UniBoxNP, image_data)
    if len(sig) == len(rad)+1:
        sig = np.delete(sig,-1)
    plt.plot(rad, sig, linewidth=0.4, color=p(i/(n-1)), label=str(i+1))
    # plt.plot(rad, sig, linewidth=0.4, color=p(i/(n-1)), label=i)
plt.title('Signal to noise on a (random) line through the star')
plt.xlim(-65, 65)
plt.xlabel('Distance from star [pixels] (+/- 0 is arbitrary)')
plt.ylabel('Signal strength (on a random profile cut)')
plt.legend(ncol=3)
plt.show()

# Find angular and radial profile cuts on a single WLD

In [ ]:
# Use brute force to find the angle between the zero axis and the planet; the planet is about 43 pixels away from 100,100
signal_at_43, x, y = [], [], []
degrees = range(360)
for i in degrees:
    x.append(100 + 43*np.cos(np.deg2rad(i)))
    y.append(100 + 43*np.sin(np.deg2rad(i)))
    signal_at_43.append(signalMatrix[round(x[i]), round(y[i])]) ###
# print(x), print(y)
plt.plot(degrees, signal_at_43)
plt.title('Signal strength on a circle, r=43 pixes, around star (2020)')
plt.xlabel('$\\theta$, Angle [degrees]')
plt.ylabel('Signal')
plt.grid()
plt.show()
# the planet is 329-330 degrees 
print('maximum occurs at:', np.where(signal_at_43 == np.amax(signal_at_43)))
print('329 and 330 are:', signal_at_43[329], signal_at_43[330])
# Currently, there are repeated pixels in the circle of signal.. can remove extras, if important 
xPlanet = 100 + 43*np.cos(np.deg2rad(329.5))
yPlanet = 100 + 43*np.sin(np.deg2rad(329.5))
print('the format is such that the planet position is:', xPlanet, yPlanet) #This is what's inputted 

# Do above for 2018 observation, r ~ 23.26
signal_at_23_2018, x_2018, y_2018 = [], [], []
radius = 23.26
for i in degrees:
    x_2018.append(100 + radius*np.cos(np.deg2rad(i)))
    y_2018.append(100 + radius*np.sin(np.deg2rad(i))) #print(signal_at_23_2018) #print(i)
    signal_at_23_2018.append(signalMatrix_2018[round(x_2018[i]), round(y_2018[i])]) ###
plt.plot(degrees, signal_at_23_2018)
plt.title('Signal strength on a circle, r=23 pixes, around star (2018)'), plt.xlabel('$\\theta$, Angle [degrees]')
plt.ylabel('Signal'), plt.grid(), plt.show()
# the planet is at 331 degrees 
print('maximum occurs at:', np.where(signal_at_23_2018 == np.amax(signal_at_23_2018)))
print('331 is:', signal_at_23_2018[331]) # Currently, there are repeated pixels in the circle of signal.. can remove extras, if important 
xPlanet_2018 = 100 + radius*np.cos(np.deg2rad(331))
yPlanet_2018 = 100 + radius*np.sin(np.deg2rad(331))
print('the format is such that the planet position in 2018 is:', xPlanet_2018, yPlanet_2018)

'''
# Do above for 2016 observation, r ~ 23.26
signal_at_23_2018, x_2018, y_2018 = [], [], []
radius = 23.26
for i in degrees:
    x_2018.append(100 + radius*np.cos(np.deg2rad(i)))
    y_2018.append(100 + radius*np.sin(np.deg2rad(i))) #print(signal_at_23_2018) #print(i)
    signal_at_23_2018.append(signalMatrix_2018[round(x_2018[i]), round(y_2018[i])]) ###
plt.plot(degrees, signal_at_23_2018)
plt.title('Signal strength on a circle, r=23 pixes, around star (2018)'), plt.xlabel('$\\theta$, Angle [degrees]')
plt.ylabel('Signal'), plt.grid(), plt.show()
# the planet is _______ degrees 
print('maximum occurs at:', np.where(signal_at_23_2018 == np.amax(signal_at_23_2018)))
print('331 is:', signal_at_23_2018[331]) # Currently, there are repeated pixels in the circle of signal.. can remove extras, if important 
xPlanet_2018 = 100 + radius*np.cos(np.deg2rad(331))
yPlanet_2018 = 100 + radius*np.sin(np.deg2rad(331))
print('the format is such that the planet position in 2018 is:', xPlanet_2018, yPlanet_2018)
'''
# Do above for 2015 observation, r ~ 23.26
signal_at_31_2015, x_2015, y_2015 = [], [], []
radius = 30.805843601
for i in degrees:
    x_2015.append(100 + radius*np.cos(np.deg2rad(i)))
    y_2015.append(100 + radius*np.sin(np.deg2rad(i)))
    signal_at_31_2015.append(signalMatrix_2015[round(x_2015[i]), round(y_2015[i])]) ###
plt.plot(degrees, signal_at_31_2015)
plt.title('Signal strength on a circle, r=31 pixes, around star (2015)'), plt.xlabel('$\\theta$, Angle [degrees]')
plt.ylabel('Signal'), plt.grid(), plt.show()
# the planet is 146 degrees 
print('maximum occurs at:', np.where(signal_at_31_2015 == np.amax(signal_at_31_2015)))
print('146 and 147 are:', signal_at_31_2015[146], signal_at_31_2015[147]) # Currently, there are repeated pixels in the circle of signal.. can remove extras, if important 
xPlanet_2015 = 100 + radius*np.cos(np.deg2rad(146.5))
yPlanet_2015 = 100 + radius*np.sin(np.deg2rad(146.5))
print('the format is such that the planet position in 2015 is:', xPlanet_2015, yPlanet_2015)

In [ ]:
# This cell calculates and plots the above, except for radius instead of angle (at an angle of 329.5)
signal_along_330, x, y = [], [], []
points_of_line = range(200)
radial_step_size = 1
for i in points_of_line:
    x.append(100 + i*radial_step_size*np.cos(np.deg2rad(329.5)))
    y.append(100 + i*radial_step_size*np.sin(np.deg2rad(329.5)))
    if round(x[i]) > 199 or round(y[i]) > 199:
        break
    else:
        signal_along_330.append(signalMatrix[round(x[i]), round(y[i])])
points_of_line = points_of_line[0:len(signal_along_330)]
#print(x), print(y), print(len(points_of_line)), print(len(signal_along_330))
plt.plot(points_of_line, signal_along_330)
plt.title('Signal strength on an axial cut, $\\theta$=330 degrees, 2020')
plt.xlabel('Radius [pixels]')
plt.ylabel('Signal')
plt.grid()
plt.show()

# the planet is 329-330 degrees
planet_signal_along_330 = signal_along_330
for i in range(6):
    planet_signal_along_330[i] = 0
print('maximum occurs at:', np.where(planet_signal_along_330 == np.amax(planet_signal_along_330)))
print('signal at point 43:', signal_along_330[43])# , signal_at_43[330])
# Currently, there are repeated pixels in the circle of signal.. can remove extras, if important 
xPlanet = 100 + 43*radial_step_size*np.cos(np.deg2rad(329.5))
yPlanet = 100 + 43*radial_step_size*np.sin(np.deg2rad(329.5))
print('(the format is such that) the planet position in 2020 is:', xPlanet, yPlanet)


signal_along_331_2018, x_2018, y_2018 = [], [], []
points_of_line_2018 = range(200)
radial_step_size = 1
for i in points_of_line_2018:
    x_2018.append(100 + i*radial_step_size*np.cos(np.deg2rad(331)))
    y_2018.append(100 + i*radial_step_size*np.sin(np.deg2rad(331)))
    if round(x_2018[i]) > 199 or round(y_2018[i]) > 199:
        break
    else:
        signal_along_331_2018.append(signalMatrix_2018[round(x_2018[i]), round(y_2018[i])])
points_of_line_2018 = points_of_line_2018[0:len(signal_along_331_2018)]     #print(x), print(y), print(len(points_of_line)), print(len(signal_along_330))
plt.plot(points_of_line_2018, signal_along_331_2018)
plt.title('Signal strength on an axial cut, $\\theta$=331 degrees, 2018')
plt.xlabel('Radius [pixels]'), plt.ylabel('Signal'), plt.grid(), plt.show()
# the planet is 329-330 degrees
planet_signal_along_330_2018 = signal_along_331_2018
for i in range(12):
    planet_signal_along_330_2018[i] = 0
print('maximum occurs at:', np.where(planet_signal_along_330_2018 == np.amax(planet_signal_along_330_2018)))
print('signal at point 23:', signal_along_331_2018[23])# , signal_at_43[330]) # Currently, there are repeated pixels in the circle of signal.. can remove extras, if important 
xPlanet = 100 + 43*radial_step_size*np.cos(np.deg2rad(331))
yPlanet = 100 + 43*radial_step_size*np.sin(np.deg2rad(331))
print('(the format is such that) the planet position in 2018 is:', xPlanet_2018, yPlanet_2018)

'''signal_along_331_2016, x_2016, y_2016 = [], [], []
points_of_line_2016 = range(200)
radial_step_size = 1
for i in points_of_line_2016:
    x_2016.append(100 + i*radial_step_size*np.cos(np.deg2rad(331)))
    y_2016.append(100 + i*radial_step_size*np.sin(np.deg2rad(331)))
    if round(x_2016[i]) > 199 or round(y_2016[i]) > 199:
        break
    else:
        signal_along_331_2016.append(signalMatrix_2016[round(x_2016[i]), round(y_2016[i])])
points_of_line_2016 = points_of_line_2016[0:len(signal_along_331_2016)]     #print(x), print(y), print(len(points_of_line)), print(len(signal_along_330))
plt.plot(points_of_line_2016, signal_along_331_2016)
plt.title('Signal strength on an axial cut, $\\theta$=331 degrees, 2016')
plt.xlabel('Radius [pixels]'), plt.ylabel('Signal'), plt.grid(), plt.show()
# the planet is 329-330 degrees
planet_signal_along_331_2016 = signal_along_331_2016
for i in range(6):
    planet_signal_along_331_2016[i] = 0
print('maximum occurs at:', np.where(signal_along_331_2016 == np.amax(planet_signal_along_331_2016)))
print('signal at point 23:', signal_along_331_2016[23])# , signal_at_43[330]) # Currently, there are repeated pixels in the circle of signal.. can remove extras, if important 
xPlanet = 100 + 43*radial_step_size*np.cos(np.deg2rad(331))
yPlanet = 100 + 43*radial_step_size*np.sin(np.deg2rad(331))
print('(the format is such that) the planet position in 2016 is:', xPlanet_2016, yPlanet_2016)'''

# Now for 2015... 
signal_along_146_2015, x_2015, y_2015 = [], [], []
points_of_line_2015 = range(200)
radial_step_size = 1
for i in points_of_line_2015:
    x_2015.append(100 + i*radial_step_size*np.cos(np.deg2rad(146.5)))
    y_2015.append(100 + i*radial_step_size*np.sin(np.deg2rad(146.5)))
    if round(x_2015[i]) > 199 or round(y_2015[i]) > 199:
        break
    else:
        signal_along_146_2015.append(signalMatrix_2015[round(x_2015[i]), round(y_2015[i])])
points_of_line_2015 = points_of_line_2015[0:len(signal_along_146_2015)]     #print(x), print(y), print(len(points_of_line)), print(len(signal_along_330))
plt.plot(points_of_line_2015, signal_along_146_2015)
plt.title('Signal strength on an axial cut, $\\theta$=331 degrees, 2015')
plt.xlabel('Radius [pixels]'), plt.ylabel('Signal'), plt.grid(), plt.show()
# the planet is _______ degrees
planet_signal_along_146_2015 = signal_along_146_2015
for i in range(6):
    planet_signal_along_146_2015[i] = 0
print('maximum occurs at:', np.where(signal_along_146_2015 == np.amax(planet_signal_along_146_2015)))
print('signal at point 31:', signal_along_146_2015[31])# , signal_at_43[330]) # Currently, there are repeated pixels in the circle of signal.. can remove extras, if important 
xPlanet = 100 + 31*radial_step_size*np.cos(np.deg2rad(331))
yPlanet = 100 + 31*radial_step_size*np.sin(np.deg2rad(331))
print('(the format is such that) the planet position in 2015 is:', xPlanet_2015, yPlanet_2015)

In [ ]:
# Plot things #3 (research's goal): sdi residual shape (6 Aug... this might be useless/redundant already)
PC_max = PC_max_ASDI
PC_used= []
for i in range(3):
    for j in range(3):
        #print(i+1,j+1)
        PC_used.append([i+1, j+1])
#print(PC_used)
#PC_used = len(SN_table[j][i])    #evaluate(pipeline, 'ADI+SDI', ([1,2,3], [1,2,3]))
                                # ((1,1) (1,2) (1,3))
                                # ((2,1) (2,2) (2,3)) 
                                 # Do I need many observations now? Or just use the sum of the i+j column as te number of pcas used?

model_completeness = np.array(np.divide(PC_used, PC_max))
first_element_of_nine_arrays = []
for i in range(len(SN_table.flatten())):
    #print(i)
    if i % 39 == 0:
        #print(SN_table.flatten()[i])
        first_element_of_nine_arrays.append(SN_table.flatten()[i])
print(first_element_of_nine_arrays)
#print(model_completeness)
print(model_completeness[:,0])
#print(SN_table)
plt.plot(model_completeness[:,0], first_element_of_nine_arrays, 'o', label='($PCA_{1,2}$)=('+str(i+1)+','+str(j+1)+')') # , color=clrs[3*i+j]
plt.xlabel('Model Completeness, $\Omega$')
plt.ylabel('Signal to Noise Ratio, SNR')
plt.legend()


#This ends the research code above; and begins coding notes below

In [ ]:
# Color spectra used for plots
clrs = ['brown', 'maroon', 'red', 'tomato', 'coral', 'darkorange', 'tan', 'orange', 'wheat', 'gold', 'darkkhaki', 'lightyellow', 'olive', 'yellow', 'yellowgreen', 'chartreuse', 'darkseagreen', 'lightgreen', 'forestgreen', 'lime', 'springgreen', 'aquamarine', 'lightseagreen', 'lightcyan', 'teal', 'darkcyan', 'cyan', 'skyblue', 'slategray', 'royalblue', 'navy', 'slateblue', 'blue', 'mediumpurple', 'blueviolet', 'indigo', 'darkviolet', 'thistle', 'plum', 'purple', 'magenta', 'deeppink', 'crimson', 'pink']
greens = ['#88ff6b', '#7bff5b', '#6aff45', '#58ff30', '#45ff18', '#32ff00', '#28cf00', '#21ac00', '#1c8f00', '#167000', '#0f5000', '#092d00', '#031200']

In [ ]:
# Plot the gradations of green colors
fig = plt.figure(figsize = (12, 4))
ax = fig.add_subplot(1, 1, 1)
a = np.empty(9)
for i in range(len(greens)):
    for j in range(len(a)):
        a[j] = i
    ax.plot(range(9), a, color=greens[i], label='$green_{'+str(i)+'}$')
handles, labels = ax.get_legend_handles_labels() # This line flips the order of the legend, so it aligns with the plot's colors ###
plt.legend(handles[::-1], labels[::-1], title='Shades of green', loc='upper right') # same as line above

In [ ]:
'''        The Code Appendix


# plot HEAT MAPS of average Signal-to-Noise Ratios for the varied PCAs (14 Jun - getting formatting right)


# Sven's imshow code recommendation (for good/simple plots)
fig, axs = plt.subplot((4,4), ....)
axs[0, 0].imshow(....)
axs[0, 0].cbar....
axs[0, 1].imshow(....)





#mat_file = 'test_mat.mat'
# Now load in the data from the .mat that was just saved
#df = scipy.io.loadmat(mat_file)
# And just to check if the data is the same:  
#assert np.all(mat_file == matdata['out'])
#df = pd.read_table(path+file, header=None, delimiter='\t') # the numerical values are all of the "signal_to_noise"s
#df.to_csv(path+'SNR.csv', sep=' ')  
#for j in df:
    #print('j is: ', j)
    #dfNew = [i.split(' ', 999) for i in df]
    #dfNew = [i.split(' ', 999)[0] for i in j]
#print(dfNew)

    If good, turn the line above into an if statement (and the if statement into a function)
if df2[0][0][0] == '(' or df2[0][0][0] == ')':
    PCAs1 = df2[0][0][1]
else:
    PCAs1 = df2[0][0][0]


#plt.figure(figsize = (20, 6))
#ax = fig.add_subplot(3, 1, 1)


for i in range(PCAs1):
    for j in range(PCAs2):
        for k in range(wavelength_index):
            print(k)
            if j < 3:
                SN_table[i, j, k] = df[j][k]
            elif j == 4 or j == 5 or j == 6:
                SN_table[i, j, k] = df[j+1][k]
            elif j == 4 or j == 5 or j == 6:
                SN_table[i, j, k] = df[j+2][k]
            else:
                True
   
built_element = '' # or []? 
            if df[i][j][k] not in bad_character:   #if the character is a number
                if SN_table[i, j, k] == 0:
                    print(k)
                    print(df[i][j][k])
                    built_element = df[i][j][k]     #(unless it's the first character)
                else:
                    built_element.append(df[i][j][k])    #add it
            print('built_element is:', built_element)
            SN_table[i, j, k] = float(built_element)


for i in range(11): # 11 columns in the file
    for k in range(39): # 39 rows in the file
        if i == 0 or i == 1 or i == 2:
            SN_table[i, 0, k] = df[i][k]
        elif i == 4 or i == 5 or i == 6:
            SN_table[i-4, 1, k] = df[i][k]
        elif i == 8 or i == 9 or i == 10:
            SN_table[i-8, 2, k] = df[i][k]
        #else:
            #print('i != 0,1,2, 4,5,6, 8,9,10')
print(SN_table[0][0])




####################################################################
# sns heatmap code (use the imshow unless that's impossible)
uniform_data = np.random.rand(10, 12)
ax = sns.heatmap(uniform_data, linewidth=0.5)
plt.show()
####################################################################
df = pd.DataFrame(np.random.random((4,4)))
df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]))
vmin = min(df.values.min(), df2.values.min())
vmax = max(df.values.max(), df2.values.max())
fig, axs = plt.subplots(nrows=2, ncols=3, gridspec_kw=dict(width_ratios=[4,4,0.4]))
sns.heatmap(df, annot=True, cbar=False, ax=axs[0,0], vmin=vmin)
sns.heatmap(df, annot=True, cbar=False, ax=axs[0,1], vmin=vmin)
sns.heatmap(df2, annot=True, yticklabels=False, cbar=False, ax=axs[1,0], vmax=vmax)
sns.heatmap(df2, annot=True, yticklabels=False, cbar=False, ax=axs[1,1], vmax=vmax)
#fig.colorbar(axs[1].collections[0], cax=axs[2]) #works with 1x2, but not 2x2
plt.show()
####################################################################



for i in range(11): # 11 columns in the file
    for k in range(39): # 39 rows in the file
        if i == 0 or i == 1 or i == 2:
            SN_table_2018[i, 0, k] = df_2018[i][k]     # print('df_2018[i][k] is:', df_2018[i][k])
            SN_table_2016[i, 0, k] = df_2016[i][k]
            SN_table_2015[i, 0, k] = df_2015[i][k]
        elif i == 4 or i == 5 or i == 6:
            SN_table_2018[i-4, 1, k] = df_2018[i][k]
            SN_table_2016[i-4, 1, k] = df_2016[i][k]
            SN_table_2015[i-4, 1, k] = df_2015[i][k]
        elif i == 8 or i == 9 or i == 10:
            SN_table_2018[i-8, 2, k] = df_2018[i][k]
            SN_table_2016[i-8, 2, k] = df_2016[i][k]
            SN_table_2015[i-8, 2, k] = df_2015[i][k]
        #else:
            #print('i != 0,1,2, 4,5,6, 8,9,10')
#print(SN_table[0][0])


    if year == 2020:
        signalMatrix = signalMatrix
    elif year == 2018:
        signalMatrix = signalMatrix_2018
    elif year == 2016:
        signalMatrix = signalMatrix_2016
    elif year == 2015:
        signalMatrix = signalMatrix_2015
    else:
        print('The year is not recognized... add it or edit it, perhaps')



'''